# LoRA Demo

This demo aims to build a portable and efficient sentiment-analysis training pipeline using RoBERTa and LoRA.

The code must run seamlessly in both Google Colab and local environments by automatically loading Hugging Face tokens from Colab userdata or a local .env file.

The model requires preprocessing steps such as tokenization, adding a missing PAD token, and configuring label mappings. To reduce memory usage, the system uses parameter-efficient fine-tuning with LoRA. The goal is to train, evaluate, and compare the model’s performance before and after fine-tuning on the GLUE SST-2 dataset.

In [81]:
!pip install datasets>=4.4.1 transformers>=4.57.3 peft>=0.18.0 evaluate>=0.4.6 huggingface_hub python-dotenv>=1.0.1

In [82]:
import os

# Detect if running in Google Colab
def in_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

if in_colab():
    from google.colab import userdata
    os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN_WRITE")
else:
    # Running locally → load from .env
    from dotenv import load_dotenv
    load_dotenv()  # loads variables from .env into environment
    os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")


In [83]:
from datasets import load_dataset, Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)
from peft import LoraConfig, get_peft_model, TaskType
import torch
import evaluate
import numpy as np


In [84]:
# -----------------------------
# Device & dtype setup
# -----------------------------
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if device == "cuda" else torch.float32

print(f"Using device: {device} with dtype: {torch_dtype}")

Using device: cuda with dtype: torch.float16


In [85]:
# Load dataset
# Standard GLUE SST-2 dataset - Sentiment Analysis of given sentences
dataset =  load_dataset("glue", "sst2")
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

In [86]:
base_model_name = "FacebookAI/roberta-base"
fine_tunned_model_name = "roberta-sentiment-analysis"

# define label maps
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

# AutoConfig loads RoBERTa’s default configuration but overrides some fields:
# num_labels=2 → adds a classification head with 2 output labels
# id2label and label2id → maps between label IDs and label names
config = AutoConfig.from_pretrained(
    base_model_name,
    num_labels=2,
    id2label=id2label,
    label2id=label2id,
)

# Loads tokenizer for RoBERTa
# This tokenizer: Splits text into tokens, converts tokens to IDs, and handles special tokens like [CLS], [SEP], and [PAD].
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# RoBERTa itself is not specifically a classification model; it's a general language model.
# But when we load it with AutoModelForSequenceClassification, it becomes a classifier.
# why? Because we specify the config with num_labels=2
# how? By using AutoModelForSequenceClassification, we are telling the model to add a classification head on top of the base RoBERTa model.
# You will see an output layer added "(out_proj): Linear(in_features=768, out_features=3, bias=True)"
base_model = AutoModelForSequenceClassification.from_pretrained(
    base_model_name,
    config=config,
    device_map="auto" if device == "cuda" else None,
    torch_dtype=torch_dtype,
)

base_model.to(device)

print(f"Before adding PAD token, tokenizer vocalbulary size: {len(tokenizer)}")
print(f"Before adding PAD token, tokenizer padding token: {tokenizer.pad_token}")

# Padding is needed because transformer models (like RoBERTa, BERT, GPT)
# only work with fixed-length batches, but sentences in real life have variable lengths.
# RoBERTa does NOT have a pad token by default.
# It uses the <mask> token as padding—but this is not ideal for training.
if tokenizer.pad_token is None:
    print("Adding PAD token to tokenizer and resizing model embeddings...")
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    base_model.resize_token_embeddings(len(tokenizer))

print(f"After adding PAD token, tokenizer vocalbulary size: {len(tokenizer)}")
print(f"After adding PAD token, tokenizer padding token: {tokenizer.pad_token}")


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Before adding PAD token, tokenizer vocalbulary size: 50265
Before adding PAD token, tokenizer padding token: <pad>
After adding PAD token, tokenizer vocalbulary size: 50265
After adding PAD token, tokenizer padding token: <pad>


In [87]:
# Base Model Structure
print(base_model)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [88]:
# Create tokenization function
# this function will be applied to each record in the dataset
# it extracts the sentence, tokenizes it to IDs, and truncates/pads to max length of 512
def tokenize_function(examples):
    # extract the sentence
    sentences = examples["sentence"]
    # tokenize and truncate/pad to max length
    tokenizer.truncation_side = 'left'
    tokenized_inputs = tokenizer(
        sentences,
        return_tensors='np',
        truncation=True,
        max_length=512
    )
    return tokenized_inputs

# tokenize training and validation datasets
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets


Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [89]:
# Data collator to dynamically pad the inputs received, so they are of equal length within a batch
# Data collators are used to batch multiple samples of data together and prepare it for training.
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [90]:
# evaluate library (by Hugging Face) lets you load standard evaluation metrics.
# it caldculates accuracy by comparing predicted labels to true labels.
accuracy_metric = evaluate.load("accuracy")

# define evaluation function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_metric.compute(predictions=predictions, references=labels)}

In [91]:
# Define a list of samples for testing the tokenizer
text_list = [
    # Positive
    "I loved the new Batman movie!",
    "What an amazing experience!",
    "The service was surprisingly good, even though the restaurant was packed.",
    "Absolutely fantastic performance, though a bit too long for my taste.",
    "The concert had incredible energy, yet the sound quality was pleasing.",
    "The dessert was delightful.",
    "I loved the artwork.",
    "The new phone works well.",
    "The flight was smooth.",
    "I was thrilled by the surprise party.",

    # Negative
    "The food at that restaurant was terrible.",
    "I will never go back to that place again.",
    "I was disappointed that my favorite dish was sold out.",
    "The book was thrilling at first, but the ending left me mindblowing.",  # could be positive/negative, marking negative
    "The hotel room looked nothing like the photos online, but the staff were friendly.",  # neutral → marking negative
    "The movie had stunning visuals, but the plot was overly predictable.",
    "The customer support solved my issue quickly, though I had to wait on hold for a long time.",
    "I appreciated the thoughtful gift, but the packaging was damaged upon delivery."
]



## Test the BASE MODEL (before Finetunning)

In [92]:
print("Untrained model predictions:")
for text in text_list:
    inputs = tokenizer(text, return_tensors="pt")

    # Move inputs to same device as model
    inputs = {k: v.to(device) for k, v in inputs.items()}

    outputs = base_model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)
    print(f"Text: {text} - {id2label[predictions.item()]}")
    print()

Untrained model predictions:
Text: I loved the new Batman movie! - POSITIVE

Text: What an amazing experience! - POSITIVE

Text: The service was surprisingly good, even though the restaurant was packed. - POSITIVE

Text: Absolutely fantastic performance, though a bit too long for my taste. - POSITIVE

Text: The concert had incredible energy, yet the sound quality was pleasing. - POSITIVE

Text: The dessert was delightful. - POSITIVE

Text: I loved the artwork. - POSITIVE

Text: The new phone works well. - POSITIVE

Text: The flight was smooth. - POSITIVE

Text: I was thrilled by the surprise party. - POSITIVE

Text: The food at that restaurant was terrible. - POSITIVE

Text: I will never go back to that place again. - POSITIVE

Text: I was disappointed that my favorite dish was sold out. - POSITIVE

Text: The book was thrilling at first, but the ending left me mindblowing. - POSITIVE

Text: The hotel room looked nothing like the photos online, but the staff were friendly. - POSITIVE

T

## Finetunning using LoRA

In [93]:
# print base model size
base_model_size = sum(param.numel() for param in base_model.parameters())
print(f"Base model size: {base_model_size/1e6:.2f} million parameters")


Base model size: 124.65 million parameters


In [94]:
# Define LoRA configuration
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, # Task type = sequence classification (e.g., sentiment analysis). This tells LoRA which parts of the model to modify.
    inference_mode=False, # Set to False because you are training. True would freeze the base model for inference.
    r=4, # Rank of the low-rank decomposition. LoRA inserts small weight matrices of size r instead of modifying the full weight matrix.
    lora_alpha=32, # Scaling factor for LoRA weights (helps control magnitude).
    lora_dropout=0.1, # Dropout applied to LoRA layers during training (prevents overfitting).
    target_modules=["query"] # target_modules specifies which parts of the transformer model will get LoRA adapters.
                            # In multi-head attention, each attention layer has weights for query (Q), key (K), value (V), and output (O)
                            # By setting target_modules=["query"], LoRA will only inject trainable adapters into the query weight matrices.
                            # This reduces the number of trainable parameters even further.
)

peft_model = get_peft_model(base_model, lora_config)
peft_model.print_trainable_parameters()


trainable params: 665,858 || all params: 125,313,028 || trainable%: 0.5314


In [95]:
# # peft_model size
# peft_model_size = sum(param.numel() for param in peft_model.parameters())
# print(f"PEFT model size: {peft_model_size/1e6:.2f} million parameters")


In [96]:
# Hyperparameters
learning_rate = 2e-4
batch_size = 16
num_epochs = 3
weight_decay = 0.01

# Training Configuration
training_args = TrainingArguments(
    output_dir=f"./outputs/{fine_tunned_model_name}",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=weight_decay,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=False,
    report_to="none", #turns off WANDB reporting
)


In [97]:
# Train the model using Trainer API
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,

)
# Train the model
trainer.train()


/tmp/ipython-input-1111618688.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The model is already on multiple devices. Skipping the move to device specified in `args`.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.280200,0.233765,{'accuracy': 0.9174311926605505}
2,0.284700,0.252930,{'accuracy': 0.9128440366972477}
3,0.256600,0.233032,{'accuracy': 0.9277522935779816}


TrainOutput(global_step=12630, training_loss=0.2815699766858268, metrics={'train_runtime': 501.3775, 'train_samples_per_second': 402.984, 'train_steps_per_second': 25.191, 'total_flos': 3759436993003656.0, 'train_loss': 0.2815699766858268, 'epoch': 3.0})

In [106]:
# Evaluate the model (evaluating peft model without merging to base)
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")
# Test the fine-tuned model
print("Fine-tuned model predictions:")
for text in text_list:
    inputs = tokenizer(text, return_tensors="pt")

    # Move inputs to same device as model
    inputs = {k: v.to(device) for k, v in inputs.items()}

    outputs = peft_model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)
    print(f"Text: {text}")
    print(f"Text: {text} - {id2label[predictions.item()]}")
    print()

Evaluation results: {'eval_loss': 0.23291015625, 'eval_accuracy': {'accuracy': 0.9277522935779816}, 'eval_runtime': 1.2066, 'eval_samples_per_second': 722.7, 'eval_steps_per_second': 45.583, 'epoch': 3.0}
Fine-tuned model predictions:
Text: I loved the new Batman movie!
Text: I loved the new Batman movie! - POSITIVE

Text: What an amazing experience!
Text: What an amazing experience! - POSITIVE

Text: The service was surprisingly good, even though the restaurant was packed.
Text: The service was surprisingly good, even though the restaurant was packed. - POSITIVE

Text: Absolutely fantastic performance, though a bit too long for my taste.
Text: Absolutely fantastic performance, though a bit too long for my taste. - POSITIVE

Text: The concert had incredible energy, yet the sound quality was pleasing.
Text: The concert had incredible energy, yet the sound quality was pleasing. - POSITIVE

Text: The dessert was delightful.
Text: The dessert was delightful. - POSITIVE

Text: I loved the a

## Upload LoRA adapter to hugging face

In [ ]:
# -----------------------------
# Save & Push Only LoRA Adapter
# -----------------------------
adapter_name = "roberta-lora-adapter"
peft_model.save_pretrained(f"./{adapter_name}")
tokenizer.save_pretrained(f"./{adapter_name}")
peft_model.push_to_hub(f"mishrabp/{adapter_name}", use_auth_token=os.environ["HF_TOKEN"])
tokenizer.push_to_hub(f"mishrabp/{adapter_name}", use_auth_token=os.environ["HF_TOKEN"])

# -----------------------------
# README for LoRA Adapter
# -----------------------------
readme_content = """
---
license: mit
tags:
  - text-classification
  - sentiment-analysis
  - lora
  - peft
language: en
library_name: transformers
base_model: FacebookAI/roberta-base
datasets:
  - glue
---

# 📘 RoBERTa Sentiment Analysis — LoRA Adapter Only

This repository contains the **LoRA adapter** for fine-tuning **RoBERTa** on 2-class sentiment analysis (Negative, Positive) using GLUE SST-2 dataset.
The base model **FacebookAI/roberta-base** is NOT included, only the trainable LoRA adapters.

## 🚀 How to Use

```python
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import PeftModel
import torch

base_model_name = "FacebookAI/roberta-base"
adapter_model = "mishrabp/roberta-lora-adapter"

# Load base model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
base_model = AutoModelForSequenceClassification.from_pretrained(base_model_name, num_labels=2)

# Load LoRA adapter
model = PeftModel.from_pretrained(base_model, adapter_model)

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Example inference
texts = ["I loved the new Batman movie!", "The food was terrible."]
inputs = tokenizer(texts, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
with torch.no_grad():
    outputs = model(**inputs)
predictions = torch.argmax(outputs.logits, dim=-1)
id2label = {0:"NEGATIVE",1:"POSITIVE"}
print([id2label[i.item()] for i in predictions])

## Merge Lora Adapter to the Base Model

In [107]:
# -------------------------------
# Step 1: Merge LoRA into base model
# -------------------------------
from peft import PeftModel

# Merge and unload LoRA adapter → returns a standard AutoModelForSequenceClassification
merged_model = peft_model.merge_and_unload()

# -------------------------------
# Step 2: Save merged model locally
# -------------------------------
merged_model_name = "roberta-sentiment-analysis-merged"
merged_model.save_pretrained(merged_model_name)
tokenizer.save_pretrained(merged_model_name)

('roberta-sentiment-analysis-merged/tokenizer_config.json',
 'roberta-sentiment-analysis-merged/special_tokens_map.json',
 'roberta-sentiment-analysis-merged/vocab.json',
 'roberta-sentiment-analysis-merged/merges.txt',
 'roberta-sentiment-analysis-merged/added_tokens.json',
 'roberta-sentiment-analysis-merged/tokenizer.json')

## Upload the merge model to Hugging Face

In [108]:
# -----------------------------
# Save & push adapters to Hugging Face
# -----------------------------
merged_model.save_pretrained(f"./{merged_model_name}")
tokenizer.save_pretrained(f"./{merged_model_name}")

merged_model.push_to_hub(f"mishrabp/{merged_model_name}", use_auth_token=os.environ["HF_TOKEN"])
tokenizer.push_to_hub(f"mishrabp/{merged_model_name}", use_auth_token=os.environ["HF_TOKEN"])

/usr/local/lib/python3.12/dist-packages/transformers/utils/hub.py:917: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...-merged/model.safetensors:  13%|#3        | 33.5MB /  249MB            

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/mishrabp/roberta-sentiment-analysis-merged/commit/7e95506f68c9bc6d5ef0516c1e3375c7dbffcd57', commit_message='Upload tokenizer', commit_description='', oid='7e95506f68c9bc6d5ef0516c1e3375c7dbffcd57', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mishrabp/roberta-sentiment-analysis-merged', endpoint='https://huggingface.co', repo_type='model', repo_id='mishrabp/roberta-sentiment-analysis-merged'), pr_revision=None, pr_num=None)

In [112]:
import os
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN_WRITE")

readme_content = """
---
license: mit
tags:
  - text-classification
  - sentiment-analysis
  - sequence-classification
  - roberta
  - lora
  - peft
language: en
library_name: transformers
base_model: FacebookAI/roberta-base
datasets:
  - glue
---

# 📘 RoBERTa Sentiment Analysis — LoRA Merged Model

This repository contains a **LoRA fine-tuned RoBERTa model** for **2-class sentiment analysis** (Negative, Positive).
The base model is **`FacebookAI/roberta-base`**, and the LoRA adapters have been **merged into the base model** to produce a standalone model.

---

## 🚀 Model Overview

| Feature | Details |
|--------|---------|
| **Base Model** | `FacebookAI/roberta-base` |
| **Fine-tuning method** | LoRA (PEFT) |
| **Task** | Sentiment Classification |
| **Labels** | NEGATIVE (0), POSITIVE (1) |
| **Dataset** | GLUE SST-2 |
| **Merged** | Yes, LoRA adapter merged into base model |
| **Training Environment** | Auto-detect (Google Colab or local machine) |

---

## 🧠 How to Use the Merged Model

### 🔹 Inference Example

```python
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# -----------------------------
# Load merged model and tokenizer
# -----------------------------
model_name = "mishrabp/roberta-sentiment-analysis-merged"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Make sure model is on the correct device and in evaluation mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# -----------------------------
# Label mapping (same as training)
# -----------------------------
id2label = {0: "NEGATIVE", 1: "POSITIVE"}

# -----------------------------
# Texts for validation
# -----------------------------
text_list = [
    # Positive
    "I loved the new Batman movie!",
    "What an amazing experience!",
    "The service was surprisingly good, even though the restaurant was packed.",
    "Absolutely fantastic performance, though a bit too long for my taste.",
    "The concert had incredible energy, yet the sound quality was pleasing.",
    "The dessert was delightful.",
    "I loved the artwork.",
    "The new phone works well.",
    "The flight was smooth.",
    "I was thrilled by the surprise party.",

    # Negative
    "The food at that restaurant was terrible.",
    "I will never go back to that place again.",
    "I was disappointed that my favorite dish was sold out.",
    "The book was thrilling at first, but the ending left me mindblowing.",  # could be positive/negative, marking negative
    "The hotel room looked nothing like the photos online, but the staff were friendly.",  # neutral → marking negative
    "The movie had stunning visuals, but the plot was overly predictable.",
    "The customer support solved my issue quickly, though I had to wait on hold for a long time.",
    "I appreciated the thoughtful gift, but the packaging was damaged upon delivery."
]

# -----------------------------
# Inference
# -----------------------------
# Tokenize all texts as a batch (avoids inconsistencies and is faster)
inputs = tokenizer(
    text_list,
    return_tensors="pt",
    truncation=True,
    padding=True,
    max_length=512
)

# Move all inputs to device
inputs = {k: v.to(device) for k, v in inputs.items()}

# Run inference
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)

# Print results
for text, pred in zip(text_list, predictions):
    print(f"Text: {text}")
    print(f"Predicted Sentiment: {id2label[pred.item()]}")
    print("-" * 50)
```

---

## ⚙️ Model Details

- Merged LoRA adapters for efficient fine-tuning.
- Fully compatible with `AutoModelForSequenceClassification`.
- Trained on GLUE SST-2 dataset for 2-class sentiment analysis.

---

## 📝 Citation

```
@model{mishrabp_roberta_sentiment_analysis_merged,
  author = {Mishra, Bibhu},
  title = {RoBERTa Sentiment Analysis - LoRA Merged},
  year = 2025,
  base_model = {FacebookAI/roberta-base}
}
```

---

## 🙌 Acknowledgements

- Hugging Face Transformers
- Hugging Face PEFT
- GLUE Benchmark
- RoBERTa by Facebook AI


"""

with open("README.md", "w", encoding="utf-8") as f:
    f.write(readme_content)

from huggingface_hub import HfApi, Repository

repo_id = f"mishrabp/{merged_model_name}"

# Option 1: Using HfApi to upload README
api = HfApi()
api.upload_file(
    path_or_fileobj="README.md",
    path_in_repo="README.md",  # must be exactly README.md for HF Hub
    repo_id=repo_id,
    repo_type="model",
    token=os.environ["HF_TOKEN"]
)


CommitInfo(commit_url='https://huggingface.co/mishrabp/roberta-sentiment-analysis-merged/commit/f506d9baa314c07d4e88a802c7527d5b4ff5d21a', commit_message='Upload README.md with huggingface_hub', commit_description='', oid='f506d9baa314c07d4e88a802c7527d5b4ff5d21a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mishrabp/roberta-sentiment-analysis-merged', endpoint='https://huggingface.co', repo_type='model', repo_id='mishrabp/roberta-sentiment-analysis-merged'), pr_revision=None, pr_num=None)

## Inference (from Hugging Face)

In [111]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# -----------------------------
# Load merged model and tokenizer
# -----------------------------
model_name = "mishrabp/roberta-sentiment-analysis-merged"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Make sure model is on the correct device and in evaluation mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# -----------------------------
# Label mapping (same as training)
# -----------------------------
id2label = {0: "NEGATIVE", 1: "POSITIVE"}

# -----------------------------
# Texts for validation
# -----------------------------
text_list = [
    # Positive
    "I loved the new Batman movie!",
    "What an amazing experience!",
    "The service was surprisingly good, even though the restaurant was packed.",
    "Absolutely fantastic performance, though a bit too long for my taste.",
    "The concert had incredible energy, yet the sound quality was pleasing.",
    "The dessert was delightful.",
    "I loved the artwork.",
    "The new phone works well.",
    "The flight was smooth.",
    "I was thrilled by the surprise party.",

    # Negative
    "The food at that restaurant was terrible.",
    "I will never go back to that place again.",
    "I was disappointed that my favorite dish was sold out.",
    "The book was thrilling at first, but the ending left me mindblowing.",  # could be positive/negative, marking negative
    "The hotel room looked nothing like the photos online, but the staff were friendly.",  # neutral → marking negative
    "The movie had stunning visuals, but the plot was overly predictable.",
    "The customer support solved my issue quickly, though I had to wait on hold for a long time.",
    "I appreciated the thoughtful gift, but the packaging was damaged upon delivery."
]

# -----------------------------
# Inference
# -----------------------------
# Tokenize all texts as a batch (avoids inconsistencies and is faster)
inputs = tokenizer(
    text_list,
    return_tensors="pt",
    truncation=True,
    padding=True,
    max_length=512
)

# Move all inputs to device
inputs = {k: v.to(device) for k, v in inputs.items()}

# Run inference
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)

# Print results
for text, pred in zip(text_list, predictions):
    print(f"Text: {text}")
    print(f"Predicted Sentiment: {id2label[pred.item()]}")
    print("-" * 50)


Text: I loved the new Batman movie!
Predicted Sentiment: POSITIVE
--------------------------------------------------
Text: What an amazing experience!
Predicted Sentiment: POSITIVE
--------------------------------------------------
Text: The service was surprisingly good, even though the restaurant was packed.
Predicted Sentiment: POSITIVE
--------------------------------------------------
Text: Absolutely fantastic performance, though a bit too long for my taste.
Predicted Sentiment: POSITIVE
--------------------------------------------------
Text: The concert had incredible energy, yet the sound quality was pleasing.
Predicted Sentiment: POSITIVE
--------------------------------------------------
Text: The dessert was delightful.
Predicted Sentiment: POSITIVE
--------------------------------------------------
Text: I loved the artwork.
Predicted Sentiment: POSITIVE
--------------------------------------------------
Text: The new phone works well.
Predicted Sentiment: POSITIVE
-------